In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
train = pd.read_csv("./labeledTrainData.tsv",delimiter="\t",quoting=3)
test = pd.read_csv("./testData.tsv",delimiter="\t",quoting=3)

In [ ]:
train.head()

In [ ]:
train['review'][3]

In [ ]:
import nltk
import string

Убираю знаки пунктуации, тэги и привожу слова к их леммам

In [ ]:
from nltk.corpus import stopwords
stopwords=stopwords.words("english")

tokenizer = nltk.WordPunctTokenizer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


def clean_tweets(raw_review):
    # токенизируем
    line = tokenizer.tokenize(raw_review.lower())
    
    # уберем маленькие слова
    filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
    
    # удалим стоп слова
    good_line = [word for word in filtered_line if word not in stopwords]
    
    # лемматизация
    good_line = [wordnet_lemmatizer.lemmatize(word,'v') for word in good_line]
    
    return " ".join(good_line)
    

In [ ]:
train["clean_review"] = train.loc[:, "review"].apply(clean_tweets)
test["clean_review"] = test.loc[:, "review"].apply(clean_tweets)

In [ ]:
train['clean_review'][0]

Снова разделяю на слова

In [ ]:
train_sents = [i.split() for i in train['clean_review']]
test_sents = [i.split() for i in test['clean_review']]

Пробовал предобученные эмбеддинги, давали около 0.8

In [ ]:
import gensim
import gensim.downloader as api

#word2vec = api.load("glove-twitter-100")

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=train_sents + test_sents,
                 min_count=20,
                 window=5,
                 sample=6e-5,
                 vector_size=300,
                 workers=4)

Получаю вектор эмбэддинг для предложения, пропуски заполняю нулевыми векторами, также пробовал просто пропускать, качество примерно одинаковое.

In [ ]:
X = []
y = []
for i, sent in enumerate(train_sents):
    y.append(train['sentiment'][i])
    vectors = []
    for word in sent:
        if word in word2vec.wv:
            vectors.append(word2vec.wv[word])
        else:
            vectors.append(list(np.zeros(300)))
    X.append(np.mean(vectors, axis=0))


In [ ]:
X[0].shape

Делю на трейн и валидацию

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report

Обучаю

In [ ]:
model = SGDClassifier(alpha=0.00007, l1_ratio=0.15)
model.fit(X_train,y_train)


y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred, digits=4))

Получаю эмбэддинги предложений для тестовой выборки

In [ ]:
X_sub = []
ids = []
for i, sent in enumerate(test_sents):
    ids.append(test['id'][i].replace('"', ''))
    vectors = []
    for word in sent:
        if word in word2vec.wv:
            vectors.append(word2vec.wv[word])
        else:
            vectors.append(list(np.zeros(300)))
    X_sub.append(np.mean(vectors, axis=0))

Сабмит

In [ ]:
ans = model.predict(X_sub)

In [ ]:
pd.DataFrame({'id': ids, 'sentiment': ans}).to_csv('sub_7.csv', index=False)